<a href="https://colab.research.google.com/github/hspgit/IR-Project-Text-summarization/blob/main/new_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json

target_summaries_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/target.json"

# Fetch the JSON data from the URL
response = requests.get(target_summaries_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
target_summaries = json.loads(response.text)

# Now you can access the data within the target_summaries dictionary
print(target_summaries)  # Print the loaded JSON data
print(type(target_summaries))

In [ ]:
news_urls_json_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/news_urls.json"
response = requests.get(news_urls_json_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
news_urls = json.loads(response.text)
print(news_urls)
print(type(news_urls))

In [ ]:
article_texts_json_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/article_texts.json"
response = requests.get(article_texts_json_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
article_texts = json.loads(response.text)
print(article_texts)
print(type(article_texts))